In [1]:
# 把学生的平均分计算出来

In [37]:
import jutil
import pandas as pd
import numpy as np

## 学院分析

In [2]:
def load_college_students_df(college_code):
    sql = "select *,CONVERT(VARCHAR,birthday,112) as birth from view_stu_course_mark where college_code='%s' and grade='2014'" % college_code
    return jutil.load_pd_df(sql)

In [75]:
# 信息学院
df = load_college_students_df('04')
df.head()

,birth,birthday,city,class_code,class_name,college_code,college_name,course_code,course_name,course_type,...,province,sex,speciality_code,speciality_name,student_id,teach_class_id,teacher,term_id,term_name,term_order
0,19960615,834768000000,北京,0403201403,通信2014-03班,04,信息科学与技术学院,0471006,计算机组成原理C,限,...,北京,男,0403,通信工程,20122169,2,唐慧佳,71,2014-2015第1学期,1
1,19960615,834768000000,北京,0403201403,通信2014-03班,04,信息科学与技术学院,0471006,计算机组成原理C,限,...,北京,男,0403,通信工程,20122169,2,唐慧佳,73,2014-2015第1学期补考,1
2,19960615,834768000000,北京,0403201403,通信2014-03班,04,信息科学与技术学院,1271031,概率论与数理统计B,必,...,北京,男,0403,通信工程,20122169,5,李裕奇,78,2015-2016第2学期补考,3
3,19960615,834768000000,北京,0403201403,通信2014-03班,04,信息科学与技术学院,3006655,电路分析BⅡ,必,...,北京,男,0403,通信工程,20122169,2,郭爱,79,2016-2017第1学期,3
4,19960615,834768000000,北京,0403201403,通信2014-03班,04,信息科学与技术学院,3133000,现代通信原理,必,...,北京,男,0403,通信工程,20122169,1,郝莉,73,2014-2015第1学期补考,1


In [78]:
# 要根据学生id聚类然后计算其平均分
def get_avg_stu_marks_df(df):
    groupd  = df.groupby(['student_id','college_code','college_name','sex','name','birth','class_code','class_name','nation_name',
        'party','province','city','speciality_code','speciality_name','grade','constellation'],as_index=False) # 保留列
    groupd = groupd.agg({'pmark':np.average})
    return groupd

In [79]:
marks = get_avg_stu_marks_df(df)
marks.head()

,student_id,college_code,college_name,sex,name,birth,class_code,class_name,nation_name,party,province,city,speciality_code,speciality_name,grade,constellation,pmark
0,20122169,04,信息科学与技术学院,男,王钧毅,19960615,0403201403,通信2014-03班,汉族,中国共产主义青年团团员,北京,北京,0403,通信工程,2014,双子座,53.827358
1,20122639,04,信息科学与技术学院,男,杨永虎,19920702,0408201402,软件2014-02班,汉族,中国共产党预备党员,山东,聊城市,0408,软件工程,2014,巨蟹座,84.775641
2,20130068,04,信息科学与技术学院,男,任添笑,19950615,0402201401,计算机2014-01班,汉族,中国共产主义青年团团员,陕西,西安市,0402,计算机科学与技术,2014,双子座,81.977778
3,20131649,04,信息科学与技术学院,男,王鑫,19940202,0406201401,自动2014-01班,汉族,中国共产主义青年团团员,重庆,重庆市,0406,自动化,2014,水瓶座,61.821591
4,20131664,04,信息科学与技术学院,男,盛守彬,19940208,0406201402,自动2014-02班,汉族,中国共产主义青年团团员,辽宁,本溪市,0406,自动化,2014,水瓶座,55.774000


In [31]:
sex_cnt = marks['sex'].value_counts().to_dict()
type(int(sex_cnt['男']))

int

In [34]:
# 计算学院男女的平均成绩
g = marks.groupby('sex',as_index=False).agg({'pmark':np.average})
dg = dict(g)
type(dg['pmark'][0].item())

float

In [36]:
type(marks['pmark'].mean().item())

float

In [24]:
# 计算学院学霸和学渣人数
g = marks[marks['pmark']>90].groupby('sex',as_index=False).agg({'pmark':'count'}).to_dict()
# dict(g)
len(g['sex'])

2

In [23]:
g = marks[marks['pmark']<65].groupby('sex',as_index=False).agg({'pmark':'count'}).to_dict()

In [25]:
# 计算学院挂科人数
g = df[df['pmark']<60].drop_duplicates(subset=['student_id']).groupby('sex',as_index=False).agg({'pmark':'count'}).to_dict()
g

{'pmark': {0: 74, 1: 342}, 'sex': {0: '女', 1: '男'}}

## 特别条件成绩分析

## 按星座

In [59]:
def get_constellation(row):
    birth = str(row['birth'])[4:]
    if (birth>='1222' and birth<='1231') or (birth>='0101' and birth<='0119') :
        return '魔羯座'
    elif birth>='0120' and birth<='0218':
        return '水瓶座'
    elif birth>='0217' and birth<='0320':
        return '双鱼座'
    elif birth>='0319' and birth<='0420':
        return '白羊座'
    elif birth>='0419' and birth<='0520':
        return '金牛座'
    elif birth>='0519' and birth<='0621':
        return '双子座'
    elif birth>='0620' and birth<='0722':
        return '巨蟹座'
    elif birth>='0721' and birth<='0822':
        return '狮子座'
    elif birth>='0821' and birth<='0922':
        return '处女座'
    elif birth>='0921' and birth<='1022':
        return '天秤座'
    elif birth>='1023' and birth<='1121':
        return '天蝎座'
    elif birth>='1122' and birth<='1221':
        return '射手座'
    else:
        return '未知星座'

In [77]:
# 按星座
df['constellation'] = df.apply(get_constellation,axis=1)
df.head()

,birth,birthday,city,class_code,class_name,college_code,college_name,course_code,course_name,course_type,...,sex,speciality_code,speciality_name,student_id,teach_class_id,teacher,term_id,term_name,term_order,constellation
0,19960615,834768000000,北京,0403201403,通信2014-03班,04,信息科学与技术学院,0471006,计算机组成原理C,限,...,男,0403,通信工程,20122169,2,唐慧佳,71,2014-2015第1学期,1,双子座
1,19960615,834768000000,北京,0403201403,通信2014-03班,04,信息科学与技术学院,0471006,计算机组成原理C,限,...,男,0403,通信工程,20122169,2,唐慧佳,73,2014-2015第1学期补考,1,双子座
2,19960615,834768000000,北京,0403201403,通信2014-03班,04,信息科学与技术学院,1271031,概率论与数理统计B,必,...,男,0403,通信工程,20122169,5,李裕奇,78,2015-2016第2学期补考,3,双子座
3,19960615,834768000000,北京,0403201403,通信2014-03班,04,信息科学与技术学院,3006655,电路分析BⅡ,必,...,男,0403,通信工程,20122169,2,郭爱,79,2016-2017第1学期,3,双子座
4,19960615,834768000000,北京,0403201403,通信2014-03班,04,信息科学与技术学院,3133000,现代通信原理,必,...,男,0403,通信工程,20122169,1,郝莉,73,2014-2015第1学期补考,1,双子座


In [74]:
# 按星座性别求平均分和人数
marks.groupby(['constellation','sex'],as_index=False).agg({'pmark':'mean','grade':'count'}).rename(columns={'pmark':'mean','grade':'count'}).to_dict()

{'constellation': {0: '双子座',
  1: '双子座',
  2: '双鱼座',
  3: '双鱼座',
  4: '处女座',
  5: '处女座',
  6: '天秤座',
  7: '天秤座',
  8: '天蝎座',
  9: '天蝎座',
  10: '射手座',
  11: '射手座',
  12: '巨蟹座',
  13: '巨蟹座',
  14: '水瓶座',
  15: '水瓶座',
  16: '牡羊座',
  17: '牡羊座',
  18: '狮子座',
  19: '狮子座',
  20: '金牛座',
  21: '金牛座',
  22: '魔羯座',
  23: '魔羯座'},
 'count': {0: 8,
  1: 53,
  2: 22,
  3: 35,
  4: 16,
  5: 59,
  6: 22,
  7: 43,
  8: 20,
  9: 45,
  10: 7,
  11: 39,
  12: 8,
  13: 38,
  14: 13,
  15: 46,
  16: 15,
  17: 33,
  18: 18,
  19: 47,
  20: 12,
  21: 38,
  22: 15,
  23: 41},
 'mean': {0: 80.83506024293183,
  1: 77.65087017555014,
  2: 80.19335151773625,
  3: 77.69513043681826,
  4: 83.50588314338069,
  5: 76.37534502627447,
  6: 83.68700635702194,
  7: 76.99323117445243,
  8: 83.01538086458548,
  9: 77.98285247501643,
  10: 80.07246017716851,
  11: 77.72793223787006,
  12: 83.1719018966582,
  13: 74.35539383517879,
  14: 80.27281307042855,
  15: 75.58587463634089,
  16: 78.92020979228958,
  17: 77.347691976749

In [56]:
# 按星座求学霸人数
marks[marks['pmark']>90].groupby(['constellation','sex'],as_index=False).agg({'pmark':'count'}).rename(columns={'pmark':'count'})

,constellation,sex,count
0,双子座,男,1
1,处女座,男,1
2,天秤座,女,1
3,天秤座,男,1
4,天蝎座,女,1
5,天蝎座,男,1
6,牡羊座,男,1
7,狮子座,女,1


In [57]:
# 按星座求学渣人数
marks[marks['pmark']<65].groupby(['constellation','sex'],as_index=False).agg({'pmark':'count'}).rename(columns={'pmark':'count'})

,constellation,sex,count
0,双子座,男,2
1,双鱼座,男,2
2,处女座,男,5
3,天秤座,男,4
4,天蝎座,男,4
5,射手座,男,2
6,巨蟹座,男,3
7,水瓶座,男,4
8,牡羊座,男,5
9,狮子座,男,4


In [80]:
# 按星座求挂科人数
df[df['pmark'] < 60].drop_duplicates(subset=['student_id']) \
    .groupby(['constellation','sex'],as_index=False).agg({'pmark':'count'}).rename(columns={'pmark':'count'})

,constellation,sex,count
0,双子座,女,4
1,双子座,男,36
2,双鱼座,女,14
3,双鱼座,男,19
4,处女座,女,7
5,处女座,男,42
6,天秤座,女,8
7,天秤座,男,30
8,天蝎座,女,5
9,天蝎座,男,27


## 按民族

In [60]:
# 按民族性别求平均分和人数
marks.groupby(['nation_name','sex'],as_index=False).agg({'pmark':'mean','grade':'count'}).rename(columns={'pmark':'mean','grade':'count'})

,nation_name,sex,mean,count
0,侗族,女,70.863154,2
1,侗族,男,78.447442,2
2,傣族,男,70.746559,1
3,哈萨克族,女,68.457500,1
4,哈萨克族,男,54.602532,1
5,回族,女,77.793312,2
6,回族,男,72.765940,3
7,土家族,女,82.401426,2
8,土家族,男,81.696900,4
9,壮族,女,77.215595,2


In [61]:
# 按民族求学霸人数
marks[marks['pmark']>90].groupby(['nation_name','sex'],as_index=False).agg({'pmark':'count'}).rename(columns={'pmark':'count'})

,nation_name,sex,count
0,汉族,女,3
1,汉族,男,5


In [62]:
# 按民族求学渣人数
marks[marks['pmark']<65].groupby(['nation_name','sex'],as_index=False).agg({'pmark':'count'}).rename(columns={'pmark':'count'})

,nation_name,sex,count
0,哈萨克族,男,1
1,外籍,男,1
2,布依族,男,1
3,汉族,男,34
4,藏族,男,1
5,黎族,男,1


In [64]:
# 按民族求挂科人数
marks[marks['pmark']<60].groupby(['nation_name','sex'],as_index=False).agg({'pmark':'count'}).rename(columns={'pmark':'count'})

,nation_name,sex,count
0,哈萨克族,男,1
1,外籍,男,1
2,布依族,男,1
3,汉族,男,22


## 按省份

In [53]:
marks.groupby('province',as_index=False).agg({'pmark':'mean','city':'count'}).rename(columns={'pmark':'mean','city':'count'}).head()

,province,mean,count
0,上海,74.507950,4
1,云南,72.813413,14
2,内蒙古,73.494827,7
3,北京,70.552434,10
4,北京市,87.236585,1


In [65]:
# 按省份求学霸人数
marks[marks['pmark']>90].groupby(['province','sex'],as_index=False).agg({'pmark':'count'}).rename(columns={'pmark':'count'})

,province,sex,count
0,安徽,女,1
1,山东,男,2
2,广东,男,1
3,江苏,女,1
4,浙江,女,1
5,浙江,男,2


In [66]:
# 按省份求学渣人数
marks[marks['pmark']<65].groupby(['province','sex'],as_index=False).agg({'pmark':'count'}).rename(columns={'pmark':'count'})

,province,sex,count
0,云南,男,3
1,北京,男,3
2,吉林,男,2
3,四川,男,8
4,安徽,男,1
5,安提瓜和巴布达,男,1
6,山东,男,1
7,山西,男,2
8,新疆,男,1
9,河北,男,1


In [67]:
# 按省份求挂科人数
marks[marks['pmark']<60].groupby(['province','sex'],as_index=False).agg({'pmark':'count'}).rename(columns={'pmark':'count'})

,province,sex,count
0,云南,男,3
1,北京,男,3
2,吉林,男,1
3,四川,男,7
4,安提瓜和巴布达,男,1
5,山西,男,1
6,新疆,男,1
7,河北,男,1
8,湖南,男,1
9,贵州,男,3


In [73]:
sql = "select *,CONVERT(VARCHAR,birthday,112) as birth from view_stu_course_mark " \
                  "where speciality_code='0403' and grade='2014'"
df = jutil.load_pd_df(sql)
len(df)

13623